# Paper Summary Generation Test (Bento)

This notebook tests the summary generation using the Gemini API via wearables-ape.io.

**Important:** This version downloads and extracts text from the PDF to avoid hallucination.

In [ ]:
# Configuration - Change these as needed
access_token = "38d70bc8-4a1f-4364-b789-bc2f99e6555a"
MODEL = "gemini-2.0-flash"  # Options: gemini-2.0-flash, gemini-1.5-flash, gemini-1.5-pro
MAX_PDF_CHARS = 100000  # Limit to avoid token limits

In [ ]:
# Install required packages
!pip install PyPDF2 -q

In [ ]:
# Setup: imports and API call function
import io
import json
import requests
from bento import authenticated_fwdproxy
import PyPDF2

def download_pdf_text(pdf_url: str, max_chars: int = MAX_PDF_CHARS) -> str:
    """Download a PDF and extract its text content."""
    print(f"Downloading PDF from: {pdf_url}")

    response = requests.get(pdf_url, timeout=60)
    if response.status_code != 200:
        raise Exception(f"Failed to download PDF: HTTP {response.status_code}")

    pdf_file = io.BytesIO(response.content)
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    text_parts = []
    total_chars = 0

    for page_num, page in enumerate(pdf_reader.pages):
        page_text = page.extract_text()
        if page_text:
            text_parts.append(f"--- Page {page_num + 1} ---\n{page_text}")
            total_chars += len(page_text)
            if total_chars >= max_chars:
                print(f"  Reached {max_chars} char limit at page {page_num + 1}")
                break

    full_text = "\n\n".join(text_parts)
    print(f"  Extracted {len(full_text)} characters from {len(text_parts)} pages")
    return full_text[:max_chars]

def call_gemini(prompt: str, model: str = MODEL) -> str:
    """Call Gemini API via wearables-ape.io."""
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
        "accept": "application/json",
    }

    payload = {
        "name": "llm-text-gen",
        "outputVariableName": "last_output",
        "model_api_name": model,
        "stream": False,
        "user": prompt,
    }

    with authenticated_fwdproxy() as session:
        response = session.post(
            "https://api.wearables-ape.io/conversations/sync",
            headers=headers,
            json=payload,
            timeout=180
        )

    if response.status_code != 200:
        raise Exception(f"API Error: {response.status_code}\n{response.text}")

    return response.json().get("result", response.text)

print("Functions defined successfully!")

## Test Simple API Call

In [ ]:
# Test with a simple prompt
print(f"Using model: {MODEL}")
result = call_gemini("Say hello in one sentence.")
print("Response:", result)

## Load Prompt Template

In [ ]:
# Load the prompt template and JSON files
from pathlib import Path

script_dir = Path.cwd()
if not (script_dir / "prompt.txt").exists():
    script_dir = Path("/data/users/lunadong/fbsource/fbcode/assistant/research/paper-agent/auto_summary")

# Load prompt template
with open(script_dir / "prompt.txt", "r") as f:
    prompt_template = f.read()

# Load and replace <json_template>
with open(script_dir / "summary_template.json", "r") as f:
    json_template = f.read()
prompt_template = prompt_template.replace("<json_template>", json_template)

# Load and replace <json_example>
with open(script_dir / "summary_example.json", "r") as f:
    json_example = f.read()
prompt_template = prompt_template.replace("<json_example>", json_example)

print(f"Prompt template loaded ({len(prompt_template)} characters)")

## Generate Paper Summary

**Change the `pdf_url` below to summarize a different paper.**

In [ ]:
# Set the PDF URL to summarize
pdf_url = "https://arxiv.org/pdf/2504.01018"

# Download and extract PDF text
pdf_text = download_pdf_text(pdf_url)

print(f"\nFirst 1000 characters of extracted text:")
print(pdf_text[:1000])

In [ ]:
# Build prompt with actual PDF content
prompt = prompt_template.replace("<PDF_URL>", pdf_url)

# Insert PDF content into prompt
pdf_content_section = f"\n\n========================\nPaper Content (extracted from PDF)\n========================\n\n{pdf_text}\n"
prompt = prompt.replace(
    "For the above paper in the given link,",
    f"{pdf_content_section}\nFor the above paper content,",
)

print(f"Total prompt length: {len(prompt)} characters")
print(f"Using model: {MODEL}")
print("Generating summary... (this may take 1-2 minutes)")

# Call the API
response_text = call_gemini(prompt, model=MODEL)

print(f"\nResponse received ({len(response_text)} characters)")

In [ ]:
# Parse the JSON response

# Find JSON in response (may be wrapped in markdown code blocks)
json_start = response_text.find("{")
json_end = response_text.rfind("}") + 1

if json_start != -1 and json_end > json_start:
    json_str = response_text[json_start:json_end]
    summary = json.loads(json_str)
    print("Successfully parsed JSON summary!")
    print("\n" + "="*60)
    print(json.dumps(summary, indent=2))
else:
    print("Could not find JSON in response")
    print(response_text)

In [ ]:
# Save to file
output_path = script_dir / "summary_output.json"
with open(output_path, "w") as f:
    json.dump(summary, f, indent=2)
print(f"Summary saved to: {output_path}")